In [ ]:
# imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import pycountry
import matplotlib as mpl
import geopandas as gpd
import geoplot as gplot
import gc

### Countries list

In [ ]:
# got this off from https://www.sitekickr.com/snippets/python/list-countries
countries_list = [
    ('US', 'United States'),
    ('AF', 'Afghanistan'),
    ('AL', 'Albania'),
    ('DZ', 'Algeria'),
    ('AS', 'American Samoa'),
    ('AD', 'Andorra'),
    ('AO', 'Angola'),
    ('AI', 'Anguilla'),
    ('AQ', 'Antarctica'),
    ('AG', 'Antigua And Barbuda'),
    ('AR', 'Argentina'),
    ('AM', 'Armenia'),
    ('AW', 'Aruba'),
    ('AU', 'Australia'),
    ('AT', 'Austria'),
    ('AZ', 'Azerbaijan'),
    ('BS', 'Bahamas'),
    ('BH', 'Bahrain'),
    ('BD', 'Bangladesh'),
    ('BB', 'Barbados'),
    ('BY', 'Belarus'),
    ('BE', 'Belgium'),
    ('BZ', 'Belize'),
    ('BJ', 'Benin'),
    ('BM', 'Bermuda'),
    ('BT', 'Bhutan'),
    ('BO', 'Bolivia'),
    ('BA', 'Bosnia And Herzegowina'),
    ('BW', 'Botswana'),
    ('BV', 'Bouvet Island'),
    ('BR', 'Brazil'),
    ('BN', 'Brunei Darussalam'),
    ('BG', 'Bulgaria'),
    ('BF', 'Burkina Faso'),
    ('BI', 'Burundi'),
    ('KH', 'Cambodia'),
    ('CM', 'Cameroon'),
    ('CA', 'Canada'),
    ('CV', 'Cape Verde'),
    ('KY', 'Cayman Islands'),
    ('CF', 'Central African Rep'),
    ('TD', 'Chad'),
    ('CL', 'Chile'),
    ('CN', 'China'),
    ('CX', 'Christmas Island'),
    ('CC', 'Cocos Islands'),
    ('CO', 'Colombia'),
    ('KM', 'Comoros'),
    ('CG', 'Congo'),
    ('CK', 'Cook Islands'),
    ('CR', 'Costa Rica'),
    ('CI', 'Cote D`ivoire'),
    ('HR', 'Croatia'),
    ('CU', 'Cuba'),
    ('CY', 'Cyprus'),
    ('CZ', 'Czech Republic'),
    ('DK', 'Denmark'),
    ('DJ', 'Djibouti'),
    ('DM', 'Dominica'),
    ('DO', 'Dominican Republic'),
    ('TP', 'East Timor'),
    ('EC', 'Ecuador'),
    ('EG', 'Egypt'),
    ('SV', 'El Salvador'),
    ('GQ', 'Equatorial Guinea'),
    ('ER', 'Eritrea'),
    ('EE', 'Estonia'),
    ('ET', 'Ethiopia'),
    ('FK', 'Falkland Islands (Malvinas)'),
    ('FO', 'Faroe Islands'),
    ('FJ', 'Fiji'),
    ('FI', 'Finland'),
    ('FR', 'France'),
    ('GF', 'French Guiana'),
    ('PF', 'French Polynesia'),
    ('TF', 'French S. Territories'),
    ('GA', 'Gabon'),
    ('GM', 'Gambia'),
    ('GE', 'Georgia'),
    ('DE', 'Germany'),
    ('GH', 'Ghana'),
    ('GI', 'Gibraltar'),
    ('GR', 'Greece'),
    ('GL', 'Greenland'),
    ('GD', 'Grenada'),
    ('GP', 'Guadeloupe'),
    ('GU', 'Guam'),
    ('GT', 'Guatemala'),
    ('GN', 'Guinea'),
    ('GW', 'Guinea-bissau'),
    ('GY', 'Guyana'),
    ('HT', 'Haiti'),
    ('HN', 'Honduras'),
    ('HK', 'Hong Kong'),
    ('HU', 'Hungary'),
    ('IS', 'Iceland'),
    ('IN', 'India'),
    ('ID', 'Indonesia'),
    ('IR', 'Iran'),
    ('IQ', 'Iraq'),
    ('IE', 'Ireland'),
    ('IL', 'Israel'),
    ('IT', 'Italy'),
    ('JM', 'Jamaica'),
    ('JP', 'Japan'),
    ('JO', 'Jordan'),
    ('KZ', 'Kazakhstan'),
    ('KE', 'Kenya'),
    ('KI', 'Kiribati'),
    ('KP', 'North Korea'),
    ('KR', 'South Korea'),
    ('KW', 'Kuwait'),
    ('KG', 'Kyrgyzstan'),
    ('LA', 'Laos'),
    ('LV', 'Latvia'),
    ('LB', 'Lebanon'),
    ('LS', 'Lesotho'),
    ('LR', 'Liberia'),
    ('LY', 'Libya'),
    ('LI', 'Liechtenstein'),
    ('LT', 'Lithuania'),
    ('LU', 'Luxembourg'),
    ('MO', 'Macau'),
    ('MK', 'Macedonia'),
    ('MG', 'Madagascar'),
    ('MW', 'Malawi'),
    ('MY', 'Malaysia'),
    ('MV', 'Maldives'),
    ('ML', 'Mali'),
    ('MT', 'Malta'),
    ('MH', 'Marshall Islands'),
    ('MQ', 'Martinique'),
    ('MR', 'Mauritania'),
    ('MU', 'Mauritius'),
    ('YT', 'Mayotte'),
    ('MX', 'Mexico'),
    ('FM', 'Micronesia'),
    ('MD', 'Moldova'),
    ('MC', 'Monaco'),
    ('MN', 'Mongolia'),
    ('MS', 'Montserrat'),
    ('MA', 'Morocco'),
    ('MZ', 'Mozambique'),
    ('MM', 'Myanmar'),
    ('NA', 'Namibia'),
    ('NR', 'Nauru'),
    ('NP', 'Nepal'),
    ('NL', 'Netherlands'),
    ('AN', 'Netherlands Antilles'),
    ('NC', 'New Caledonia'),
    ('NZ', 'New Zealand'),
    ('NI', 'Nicaragua'),
    ('NE', 'Niger'),
    ('NG', 'Nigeria'),
    ('NU', 'Niue'),
    ('NF', 'Norfolk Island'),
    ('MP', 'Northern Mariana Islands'),
    ('NO', 'Norway'),
    ('OM', 'Oman'),
    ('PK', 'Pakistan'),
    ('PW', 'Palau'),
    ('PA', 'Panama'),
    ('PG', 'Papua New Guinea'),
    ('PY', 'Paraguay'),
    ('PE', 'Peru'),
    ('PH', 'Philippines'),
    ('PN', 'Pitcairn'),
    ('PL', 'Poland'),
    ('PT', 'Portugal'),
    ('PR', 'Puerto Rico'),
    ('QA', 'Qatar'),
    ('RE', 'Reunion'),
    ('RO', 'Romania'),
    ('RU', 'Russian Federation'),
    ('RW', 'Rwanda'),
    ('KN', 'Saint Kitts And Nevis'),
    ('LC', 'Saint Lucia'),
    ('VC', 'St Vincent/Grenadines'),
    ('WS', 'Samoa'),
    ('SM', 'San Marino'),
    ('ST', 'Sao Tome'),
    ('SA', 'Saudi Arabia'),
    ('SN', 'Senegal'),
    ('SC', 'Seychelles'),
    ('SL', 'Sierra Leone'),
    ('SG', 'Singapore'),
    ('SK', 'Slovakia'),
    ('SI', 'Slovenia'),
    ('SB', 'Solomon Islands'),
    ('SO', 'Somalia'),
    ('ZA', 'South Africa'),
    ('ES', 'Spain'),
    ('LK', 'Sri Lanka'),
    ('SH', 'St. Helena'),
    ('PM', 'St.Pierre'),
    ('SD', 'Sudan'),
    ('SR', 'Suriname'),
    ('SZ', 'Swaziland'),
    ('SE', 'Sweden'),
    ('CH', 'Switzerland'),
    ('SY', 'Syrian Arab Republic'),
    ('TW', 'Taiwan'),
    ('TJ', 'Tajikistan'),
    ('TZ', 'Tanzania'),
    ('TH', 'Thailand'),
    ('TG', 'Togo'),
    ('TK', 'Tokelau'),
    ('TO', 'Tonga'),
    ('TT', 'Trinidad And Tobago'),
    ('TN', 'Tunisia'),
    ('TR', 'Turkey'),
    ('TM', 'Turkmenistan'),
    ('TV', 'Tuvalu'),
    ('UG', 'Uganda'),
    ('UA', 'Ukraine'),
    ('AE', 'United Arab Emirates'),
    ('GB', 'United Kingdom'),
    ('UY', 'Uruguay'),
    ('UZ', 'Uzbekistan'),
    ('VU', 'Vanuatu'),
    ('VA', 'Vatican City State'),
    ('VE', 'Venezuela'),
    ('VN', 'Viet Nam'),
    ('VG', 'Virgin Islands (British)'),
    ('VI', 'Virgin Islands (U.S.)'),
    ('EH', 'Western Sahara'),
    ('YE', 'Yemen'),
    ('YU', 'Yugoslavia'),
    ('ZR', 'Zaire'),
    ('ZM', 'Zambia'),
    ('ZW', 'Zimbabwe')
]

countries_d = {}
for c in countries_list:
    countries_d[c[1]] = c[0]

### Load data

#### Define some helper functions first

In [ ]:
def get_subregions(code=None):
    '''extracts country subregions if any'''
    
    res = []
    for s in list(pycountry.subdivisions.get(country_code=code)):
        res.append(s.name)
    
    return res

In [ ]:
def extract_date(line, start_year='2020'):
    ''' extract date from dataset'''
    
    MONTHS = {'January' : '01',
              'February' :'02' ,
              'March' : '03', 
              'April' : '04',
              'May' : '05',
              'June' : '06',
              'July' : '07',
              'August' : '08',
              'September' : '09',
              'October' : '10',
              'November' : '11',
              'December' : '12'}
    
    date = False
    
    for m in MONTHS:
        pattern = '^\d+\s+' + m
        if re.match(pattern,line):
            tokens = re.split('\s+',line)
            if len(tokens[0]) < 2:
                tokens[0] = '0' + tokens[0]    
            date = start_year + MONTHS[m] + tokens[0]
            
    return date

#### Ingest data and turn it into list of lines

In [ ]:
# ingest data
with open('covid19.txt','r') as f:
    # read everything in a single string
    text = f.read()

# get rid on new lines and build a list
lines = text.split('\n')

#### Parse data and attempt to extract meaningful information

In [ ]:
# init a dict to hold data, will turn it into dataframe later
parsed = {}
parsed['date'] = []
parsed['time'] = []
parsed['timestamp'] = []
parsed['country'] = []
parsed['region'] = []
parsed['new_cases'] = []
parsed['new_deaths'] = []

# parse data
# i feel the code below can be greatly improved, it is by no means a very good solution
current_date = None
for line in lines:   
    date = extract_date(line)
    if date:
        current_date = date
        continue

    country = 'unknown'
    subregion = 'unknown'
    new_cases = 0
    new_deaths = 0
    timestamp = 'unknown'
    
    for c in countries_d.keys():
        if c in line:
            country=c
            srs = get_subregions(code=countries_d[c])
            for sr in srs:
                if c == 'China':
                    sr = sr.split(' ')[0]
                    
                if sr in line:
                    subregion=sr
    
    if country == 'unknown':
        continue
    
    pattern = '^\d+:\d+'
    z = re.findall(pattern,line)
    if z:
        timestamp = z[0].split(' ')[0]
    else:
        timestamp = '00:00'
    
    line = line.replace(timestamp+': ','')
    timestamp = timestamp.replace(':','')
    #print(line)
    
    # new cases
    pattern = '\d+\s+new\s+case|\d+\s+case'
    z = re.findall(pattern,line)
    if z:
        new_cases = z[0].split(' ')[0]
        
    pattern = 'First\s\d+\scase'
    z = re.findall(pattern,line)
    if z:
        new_cases = z[0].split(' ')[1]
        
    pattern = 'First\s+case'
    z = re.findall(pattern,line)
    if z:
        new_cases = 1
    
    # deaths
    pattern = '\d+\s+new\s+death|\d+\s+death'
    z = re.findall(pattern,line)
    if z:
        new_deaths = z[0].split(' ')[0]
        
    if new_cases == 0 and new_deaths == 0:
        continue
    
    parsed['date'].append(current_date)
    parsed['country'].append(country)
    parsed['region'].append(subregion)
    parsed['timestamp'].append(timestamp)
    parsed['time'].append(str(current_date)+str(timestamp))
    parsed['new_cases'].append(new_cases)
    parsed['new_deaths'].append(new_deaths)
    
    #print('%s\t%s\t%s\t%s\t%s\t%s' %(current_date,timestamp,country,subregion, new_cases, new_deaths))

### Prepare for plotting

#### Data into dataframe

In [ ]:
# create dataframe for further use
df = pd.DataFrame(parsed)

# convert columns to numeric
df = df.astype({'new_cases' : int,
                'new_deaths' : int,
                'date' : int,
                'time' : int,
                'timestamp' : int
               })

#### Load geography maps

In [ ]:
# world
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.loc[world['name'] == 'United States of America', 'name'] = 'United States'

In [ ]:
# USA
contiguous_usa = gpd.read_file(gplot.datasets.get_path('contiguous_usa'))

#### The following 2 functions do the plotting.
Both are very similar and I guess I could have writtenjust one and made it very configurable but this keeps the code simpler. Both generate a graphics file and dump it in the current directory. Running them in a loop lets you generate frames which can be animated after with a utility like ffmpeg

In [ ]:
def render_frame(data=None,
                 map_data=None,
                 time_point=None,
                 feature='new_cases',
                 extent=(-180,-90,180,90),
                 dims=(16,8),
                 dpi=150,
                 img_type='png',
                 title_annot='new cases',
                 cmap='OrRd',
                 color_min=0,
                 color_max=4,
                 verbose=True):
    
    '''renders a single frame and dumps a graphics file'''
    
    # normalize colormap
    norm = mpl.colors.Normalize(vmin=color_min, vmax=color_max)
    cmap = mpl.cm.ScalarMappable(norm=norm, cmap=cmap).cmap
    
    # sum numbers by country, take log10
    sum_by_country = np.log10(data.loc[ data['time'] <= time_point ].groupby('country')[feature].sum())
    #sum_by_country = pd.DataFrame(sum_by_country.rename_axis('name'))
    
    # merge w/ map data
    geodata = map_data.copy()
    geodata[feature] = np.zeros(geodata.shape[0])
    for country in sum_by_country.index:
        geodata.loc[geodata['name'] == country,feature] = sum_by_country.loc[country]
    
    # plot
    fig, ax = plt.subplots( nrows=1, ncols=1, figsize=(16,8))
    
    #gplot.polyplot(map_data, ax = ax)
    gplot.choropleth(geodata,
                     hue=feature,
                     cmap=cmap,
                     ax=ax,
                     norm=norm,
                     extent=extent,
                     legend = True)
    
    # derive date and time for plot title
    str_date = str(time_point)[0:4] + '-' +str(time_point)[4:6]+ '-' +str(time_point)[6:8]
    str_time = str(time_point)[8:]
    
    plt.title('COVID19 '+ title_annot +' over time ' + str_date + ' ' + str_time)
    fname = str(time_point) + '.'+img_type
    
    # blurb the filename so we can track progress
    if verbose:
        print(fname)
        
    plt.savefig(fname, dpi=dpi)
    fig.clf()
    plt.close()

In [ ]:
def render_frame_usa(data=None,
                 map_data=None,
                 time_point=None,
                 feature='new_cases',
                 extent=(-130,25,-65,55),
                 dims=(16,8),
                 dpi=150,
                 img_type='png',
                 title_annot='new cases',
                 cmap='OrRd',
                 color_min=0,
                 color_max=3,
                 verbose=True):
    
    '''renders a single frame and dumps a graphics file'''
    
    # normalize colormap
    norm = mpl.colors.Normalize(vmin=color_min, vmax=color_max)
    cmap = mpl.cm.ScalarMappable(norm=norm, cmap=cmap).cmap
    
    # sum numbers by country, take log10
    usa = data.loc[data['country'] == 'United States']
    sum_by_state = np.log10(usa.loc[ usa['time'] <= time_point ].groupby('region')[feature].sum())
    
    # merge w/ map data
    geodata = map_data.copy()
    geodata[feature] = np.zeros(geodata.shape[0])
    for state in sum_by_state.index:
        geodata.loc[geodata['state'] == state,feature] = sum_by_state.loc[state]
    
    # plot
    fig, ax = plt.subplots( nrows=1, ncols=1, figsize=(16,8))
    
    gplot.choropleth(geodata,
                     hue=feature,
                     cmap=cmap,
                     ax=ax,
                     norm=norm,
                     extent=extent,
                     legend = True)
    
    # derive date and time for plot title
    str_date = str(time_point)[0:4] + '-' +str(time_point)[4:6]+ '-' +str(time_point)[6:8]
    str_time = str(time_point)[8:]
    
    plt.title('COVID19 USA'+ title_annot +' over time ' + str_date + ' ' + str_time)
    fname = str(time_point) + '.'+img_type
    
    # blurb the filename so we can track progress
    if verbose:
        print(fname)
        
    plt.savefig(fname, dpi=dpi)
    fig.clf()
    plt.close()

### Plot stuff

In [ ]:
# some defs to use later
feature_cases='new_cases'
title_cases = 'cumulative cases'

feature_deaths='new_deaths'
title_deaths='cumulative deaths'

#### Plot either the whole world or a subregion (like Europe for example)

In [ ]:
# Global spread
# Europe
extent_europe=(-20,30,60,80)
color_max_europe=3

# world
extent_world=(-180,-90,180,90)
color_max_world=4

# setup run
extent=extent_world
color_max=color_max_world
feature=feature_cases
title_annot=title_cases

# render over the entire timeline
dates = np.sort(df['date'].unique())
timestep=200 # this is 2 hours, not 200 minutes
for date in dates:
    for time in range(0,2500,timestep):
        # prepare the date+time number
        if time == 0:
            t = '0000'
        elif time < 1000:
            t = '0' + str(time)
        else:
            t = str(time)
            
        this_time = int(str(date) + t)
        
        # render the map 
        render_frame(data=df,
                     map_data=world,
                     time_point=this_time,
                     extent=extent,
                     color_max=color_max,
                     feature=feature,
                     title_annot=title_annot,
                     dims=(12,6))
        
        # maybe running into RAM problems because creating too many figures
        gc.collect()

#### Plot the USA per state

In [ ]:
# USA spread
# setup run
feature=feature_cases
title_annot=title_cases

# render over the entire timeline
dates = np.sort(df['date'].unique())
timestep=200 # this is 2 hours, 230 will be 2:30 hours
for date in dates:
    for time in range(0,2500,timestep):
        # prepare the date+time number
        if time == 0:
            t = '0000'
        elif time < 1000:
            t = '0' + str(time)
        else:
            t = str(time)
            
        this_time = int(str(date) + t)
        
        # render the map 
        render_frame_usa(data=df,
                         map_data=contiguous_usa,
                         time_point=this_time,
                         feature=feature,
                         title_annot=title_annot,
                         dims=(12,6))
                         
        
        # maybe running into RAM problems because creating too many figures
        gc.collect()